In [146]:
# Dependencies 
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import glob as glob

In [147]:
# Set path of html file
html_path = 'Race_Results_HTML/stpetersburg.htm'
# Read html file and store as html_content
with open(html_path, 'r') as file:
    html_content = file.read()

In [148]:
soup = BeautifulSoup(html_content, 'html.parser')

In [149]:
# Get the full-season table from HTML
season_html = soup.find(id = 'race-alltime')
season_data = []
# Loop through table html and parse to list
for row in season_html.find_all('tr'):
    row_data = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
    season_data.append(row_data)
season_df = pd.DataFrame(season_data[1:], columns=season_data[0])
season_df["Date"] = pd.to_datetime(season_df["Date"])
season_df = season_df.drop([1, 2])
season_df = season_df.drop("Avg.Speed", axis=1).drop("Winner", axis=1).drop("Car/Engine", axis=1).drop("Start Position", axis=1)
season_df.reset_index(drop=True, inplace=True)
season_df.index += 1
season_df

,Date,Track,Track Type
1,2024-03-10,Streets of St. Petersburg,Street Course
2,2024-03-24,The Thermal Club,Road Course
3,2024-04-21,Streets of Long Beach,Street Course
4,2024-04-28,Barber Motorsports Park,Road Course
5,2024-05-11,Indianapolis Motor Speedway Road Course,Road Course
6,2024-05-26,Indianapolis Motor Speedway,Oval
7,2024-06-02,Streets of Detroit,Street Course
8,2024-06-09,Road America,Road Course
9,2024-06-23,WeatherTech Raceway Laguna Seca,Road Course
10,2024-07-07,Mid-Ohio Sports Car Course,Road Course


In [150]:
# Data Cleaning specific to 2024 season
# Correct the incorrect date of Iowa race 1
season_df.at[11, 'Date'] = '2024-07-13'
# Create row for shorthand race name
season_df.at[1, 'Race'] = 'St. Pete'
season_df.at[2, 'Race'] = 'Thermal'
season_df.at[3, 'Race'] = 'Long Beach'
season_df.at[4, 'Race'] = 'Barber'
season_df.at[5, 'Race'] = 'Indy Road Course'
season_df.at[6, 'Race'] = 'Indy 500'
season_df.at[7, 'Race'] = 'Detroit'
season_df.at[8, 'Race'] = 'Road America'
season_df.at[9, 'Race'] = 'Laguna Seca'
season_df.at[10, 'Race'] = 'Mid-Ohio'
season_df.at[11, 'Race'] = 'Iowa 1'
season_df.at[12, 'Race'] = 'Iowa 2'
season_df.at[13, 'Race'] = 'Toronto'
season_df.at[14, 'Race'] = 'Gateway'
season_df.at[15, 'Race'] = 'Portland'
season_df.at[16, 'Race'] = 'Milwaukee 1'
season_df.at[17, 'Race'] = 'Milwaukee 2'
season_df.at[18, 'Race'] = 'Nashville'
season_df

,Date,Track,Track Type,Race
1,2024-03-10,Streets of St. Petersburg,Street Course,St. Pete
2,2024-03-24,The Thermal Club,Road Course,Thermal
3,2024-04-21,Streets of Long Beach,Street Course,Long Beach
4,2024-04-28,Barber Motorsports Park,Road Course,Barber
5,2024-05-11,Indianapolis Motor Speedway Road Course,Road Course,Indy Road Course
6,2024-05-26,Indianapolis Motor Speedway,Oval,Indy 500
7,2024-06-02,Streets of Detroit,Street Course,Detroit
8,2024-06-09,Road America,Road Course,Road America
9,2024-06-23,WeatherTech Raceway Laguna Seca,Road Course,Laguna Seca
10,2024-07-07,Mid-Ohio Sports Car Course,Road Course,Mid-Ohio


In [151]:
points_table_formula1 = pd.read_csv('points_systems_csv/2024_formula1_race_points_table.csv')
points_table_imsa = pd.read_csv('points_systems_csv/2024_imsa_race_points_table.csv')
points_table_imsa_qual = pd.read_csv('points_systems_csv/2024_imsa_qualifying_points_table.csv')

all_files_path = 'Race_Results_HTML/*.htm'
race_results_files = []
race_results_files = glob.glob(all_files_path)

season_races_df = pd.DataFrame()

for race_result_file in race_results_files:
# Read html file and store as html_content
    with open(race_result_file, 'r') as file:
        html_content = file.read()
    # Parse race result table html
    soup = BeautifulSoup(html_content, 'html.parser')
    race_table_html = soup.find(id = 'race-season')
# Initialize a list to contain the race results 
    race_result_data = []
# Loop through table html and parse to list
    for row in race_table_html.find_all('tr'):
        row_data = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
        race_result_data.append(row_data)

    race_result_df = pd.DataFrame(race_result_data[1:], columns=race_result_data[0])
    race_result_df['Rank'] = race_result_df['Rank'].astype('int64')
    race_result_df['Starts'] = race_result_df['Starts'].astype('int64')
    race_result_df['Laps'] = race_result_df['Laps'].astype('int64')
    race_result_df['Laps Led'] = race_result_df['Laps'].astype('int64')
    race_result_df['Points'] = race_result_df['Points'].astype('int64')
    race_result_df['Pit stop'] = race_result_df['Pit stop'].astype('int64')
    race_result_df = pd.merge(race_result_df, points_table_formula1, on='Rank', how='inner')
    race_result_df = pd.merge(race_result_df, points_table_imsa, on='Rank', how='inner')
    race_result_df = pd.merge(race_result_df, points_table_imsa_qual, on='Starts', how='inner')
    race_result_df['Points (IMSA Scoring)'] = race_result_df['Points (IMSA Scoring)']+race_result_df['Points (IMSA Scoring) - Qualifying']
    race_result_df = race_result_df.drop(columns=['Points (IMSA Scoring) - Qualifying'])
 
    race_name = soup.find(id = 'season-event-name').text
    race_date = soup.find(id = 'season-session-date').text
    race_date = datetime.strptime(race_date, '%A, %B %d, %Y').date()
    race_winner = race_result_df.loc[race_result_df['Rank'] == 1, 'Driver'].iloc[0]
    race_pole = race_result_df.loc[race_result_df['Starts'] == 1, 'Driver'].iloc[0]
    race_avg_speed = race_result_df.loc[race_result_df['Rank'] == 1, 'Avg.Speed'].iloc[0]
    race_total_time = race_result_df.loc[race_result_df['Rank'] == 1, 'Total time'].iloc[0]
    race_laps = race_result_df.loc[race_result_df['Rank'] == 1, 'Laps'].iloc[0]
    race_info = {'Date': [race_date],
            'Laps': [race_laps],
            'Winner': [race_winner],
            'Pole': [race_pole],
            'Avg. Speed': [race_avg_speed],
            'Total Time': [race_total_time],
            'Results': [race_result_df]}
    race_df = pd.DataFrame(race_info)
    race_df["Date"] = pd.to_datetime(race_df["Date"])
    race_df.reset_index(drop=True, inplace=True)
    race_df.index += 1
    season_races_df = pd.concat([season_races_df, race_df], axis=0)

season_races_df

,Date,Laps,Winner,Pole,Avg. Speed,Total Time,Results
1,2024-06-02,100,Scott Dixon,Colton Herta,78.251,02:06:07.9684,Rank Driver Car No. Starts ...
1,2024-04-21,85,Scott Dixon,Felix Rosenqvist,98.350,01:42:03.1416,Rank Driver Car No. Starts ...
1,2024-06-09,55,Will Power,Linus Lundqvist,126.154,01:45:00.0267,Rank Driver Car No. Starts ...
1,2024-07-13,250,Will Power,Scott McLaughlin,154.768,01:26:38.7472,Rank Driver Car No. Starts ...
1,2024-07-14,250,Scott McLaughlin,Colton Herta,128.098,01:44:41.1172,Rank Driver Car No. Starts ...
1,2024-03-10,100,Pato O'Ward,Josef Newgarden,96.752,01:51:37.5075,Rank Driver Car No. Starts ...
1,2024-08-25,110,Will Power,Santino Ferrucci,112.161,01:55:34.1948,Rank Driver Car No. Starts ...
1,2024-07-07,80,Pato O'Ward,Alex Palou,116.071,01:33:22.6191,Rank Driver Car No. Starts ...
1,2024-05-26,200,Josef Newgarden,Scott McLaughlin,167.763,02:58:49.4079,Rank Driver Car No. Starts ...
1,2024-05-11,85,Alex Palou,Alex Palou,117.956,01:45:27.2320,Rank Driver Car No. Starts ...


In [152]:
season_full_df = pd.merge(season_df, season_races_df, on="Date", how = "inner")
season_full_df.reset_index(drop=True, inplace=True)
season_full_df.index += 1
season_full_df

,Date,Track,Track Type,Race,Laps,Winner,Pole,Avg. Speed,Total Time,Results
1,2024-03-10,Streets of St. Petersburg,Street Course,St. Pete,100,Pato O'Ward,Josef Newgarden,96.752,01:51:37.5075,Rank Driver Car No. Starts ...
2,2024-04-21,Streets of Long Beach,Street Course,Long Beach,85,Scott Dixon,Felix Rosenqvist,98.350,01:42:03.1416,Rank Driver Car No. Starts ...
3,2024-04-28,Barber Motorsports Park,Road Course,Barber,90,Scott McLaughlin,Scott McLaughlin,106.369,01:56:45.7773,Rank Driver Car No. Starts ...
4,2024-05-11,Indianapolis Motor Speedway Road Course,Road Course,Indy Road Course,85,Alex Palou,Alex Palou,117.956,01:45:27.2320,Rank Driver Car No. Starts ...
5,2024-05-26,Indianapolis Motor Speedway,Oval,Indy 500,200,Josef Newgarden,Scott McLaughlin,167.763,02:58:49.4079,Rank Driver Car No. Starts ...
6,2024-06-02,Streets of Detroit,Street Course,Detroit,100,Scott Dixon,Colton Herta,78.251,02:06:07.9684,Rank Driver Car No. Starts ...
7,2024-06-09,Road America,Road Course,Road America,55,Will Power,Linus Lundqvist,126.154,01:45:00.0267,Rank Driver Car No. Starts ...
8,2024-06-23,WeatherTech Raceway Laguna Seca,Road Course,Laguna Seca,95,Alex Palou,Alex Palou,102.740,02:04:09.8545,Rank Driver Car No. Starts ...
9,2024-07-07,Mid-Ohio Sports Car Course,Road Course,Mid-Ohio,80,Pato O'Ward,Alex Palou,116.071,01:33:22.6191,Rank Driver Car No. Starts ...
10,2024-07-13,Iowa Speedway,Oval,Iowa 1,250,Will Power,Scott McLaughlin,154.768,01:26:38.7472,Rank Driver Car No. Starts ...


In [153]:
season_full_df.at[1, 'Cautions'] = 3
season_full_df.at[2, 'Cautions'] = 1
season_full_df.at[3, 'Cautions'] = 4
season_full_df.at[4, 'Cautions'] = 1
season_full_df.at[5, 'Cautions'] = 8
season_full_df.at[6, 'Cautions'] = 8
season_full_df.at[7, 'Cautions'] = 3
season_full_df.at[8, 'Cautions'] = 5
season_full_df.at[9, 'Cautions'] = 1
season_full_df.at[10, 'Cautions'] = 6
season_full_df.at[11, 'Cautions'] = 2
season_full_df.at[12, 'Cautions'] = 5
season_full_df.at[13, 'Cautions'] = 6
season_full_df.at[14, 'Cautions'] = 1
season_full_df.at[15, 'Cautions'] = 3
season_full_df.at[16, 'Cautions'] = 6
season_full_df.at[17, 'Cautions'] = 3
season_full_df['Cautions'] = season_full_df['Cautions'].astype('int64')
season_full_df

,Date,Track,Track Type,Race,Laps,Winner,Pole,Avg. Speed,Total Time,Results,Cautions
1,2024-03-10,Streets of St. Petersburg,Street Course,St. Pete,100,Pato O'Ward,Josef Newgarden,96.752,01:51:37.5075,Rank Driver Car No. Starts ...,3
2,2024-04-21,Streets of Long Beach,Street Course,Long Beach,85,Scott Dixon,Felix Rosenqvist,98.350,01:42:03.1416,Rank Driver Car No. Starts ...,1
3,2024-04-28,Barber Motorsports Park,Road Course,Barber,90,Scott McLaughlin,Scott McLaughlin,106.369,01:56:45.7773,Rank Driver Car No. Starts ...,4
4,2024-05-11,Indianapolis Motor Speedway Road Course,Road Course,Indy Road Course,85,Alex Palou,Alex Palou,117.956,01:45:27.2320,Rank Driver Car No. Starts ...,1
5,2024-05-26,Indianapolis Motor Speedway,Oval,Indy 500,200,Josef Newgarden,Scott McLaughlin,167.763,02:58:49.4079,Rank Driver Car No. Starts ...,8
6,2024-06-02,Streets of Detroit,Street Course,Detroit,100,Scott Dixon,Colton Herta,78.251,02:06:07.9684,Rank Driver Car No. Starts ...,8
7,2024-06-09,Road America,Road Course,Road America,55,Will Power,Linus Lundqvist,126.154,01:45:00.0267,Rank Driver Car No. Starts ...,3
8,2024-06-23,WeatherTech Raceway Laguna Seca,Road Course,Laguna Seca,95,Alex Palou,Alex Palou,102.740,02:04:09.8545,Rank Driver Car No. Starts ...,5
9,2024-07-07,Mid-Ohio Sports Car Course,Road Course,Mid-Ohio,80,Pato O'Ward,Alex Palou,116.071,01:33:22.6191,Rank Driver Car No. Starts ...,1
10,2024-07-13,Iowa Speedway,Oval,Iowa 1,250,Will Power,Scott McLaughlin,154.768,01:26:38.7472,Rank Driver Car No. Starts ...,6


,Rank,Driver,Car No.,Starts,Laps,Total time,Laps Led,Status,Points,Avg.Speed,Pit stop,Points (F1 Scoring),Points (IMSA Scoring)
0,1,Alex Palou,10,1,95,02:04:09.8545,95,Running,54,102.740,3,25,385
1,2,Colton Herta,26,4,95,02:04:11.8325,95,Running,41,102.712,3,18,348
2,3,Alexander Rossi,7,5,95,02:04:14.3681,95,Running,36,102.678,3,15,326
3,4,Romain Grosjean,77,8,95,02:04:14.6788,95,Running,32,102.673,3,12,303
4,5,Kyle Kirkwood,27,2,95,02:04:18.5313,95,Running,31,102.620,3,10,292
5,6,Scott Dixon,9,10,95,02:04:19.0049,95,Running,28,102.614,3,8,271
6,7,Will Power,12,15,95,02:04:19.8509,95,Running,26,102.602,3,6,256
7,8,Pato O'Ward,5,9,95,02:04:20.4759,95,Running,24,102.593,3,4,252
8,9,Santino Ferrucci,14,17,95,02:04:20.8570,95,Running,22,102.588,3,2,234
9,10,Marcus Ericsson,28,18,95,02:04:21.4943,95,Running,20,102.579,3,1,223
